In [21]:
def brothers(name):

    students = {
        "junaid" : {
            "age" : 21,
            "city" : "karachi",
            "color": "fair"
        },
        "Bari" : {
            "age" : 22,
            "city" : "lahore",
            "color": "very fair"
        }
    }
    return students 

In [ ]:
result = brothers("junaid")
myName = result.get(, {}).get("city")  # Now .get() works on the dictionary
print(myName)

karachi


In [4]:
from google.genai import types #types is used ahead for retry options
from google.adk.models.google_llm import Gemini
from google.adk.agents import LlmAgent
from google.adk.tools import AgentTool, google_search, ToolContext
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.code_executors import BuiltInCodeExecutor


In [15]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("Setup Complete")

Setup Complete


In [6]:
# Configuring retry options



retry_options = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[500, 429, 503,504]
)

##### Building a currency converter

- get payment method charges
- Conversion rates

In [9]:
def get_payment_method_charges(method : str) -> dict:
    # Function to get payment method charges
    '''
    Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}

    '''
    get_payment_method_charges_DB = {
        "credit_card": 0.02,
        "imps": 0.01,
        "upi": 0.0002,
        "paypal": 0.05
    }
    fee = get_payment_method_charges_DB.get(method.lower())
    if fee is not None:
        return {
            "status" : "success",
            "fee_percentage" : fee
        }
    else:
        return {
            "status" : "error",
            "error_message" : "Payment method not found"
        }   

In [10]:
def conversion_rate(base: str, target: str) ->dict:
    '''Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}

    '''
    conversion_DB = {
        "usd" :
        {
            "eur" : 105,
            "inr" : 83,
            'pkr' : 330
        }
    }

    rate = conversion_DB.get(base.lower(), {}).get(target.lower())
    if rate is not None:
        return {
            "status" : "success",
            "conversion_rate" : rate
        }
    else:
        return {
            "status": "ERROR",
            "error_Message": "Currenrcy pair not supported"
        }


    


In [45]:
currency_agent = LlmAgent(
    name = "currency_agent",
    model = Gemini(model = "gemini-2.5-flash-lite", retry_options=retry_options),
    instruction = """You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [get_payment_method_charges, conversion_rate]
)

In [46]:
runner = InMemoryRunner(agent=currency_agent)
response = await runner.run_debug("Convert 100 usd to inr using upi")


 ### Created new session: debug_session_id

User > Convert 100 usd to inr using upi


currency_agent > The final converted amount is 8298.34 INR.

Here's the breakdown:
- The fee for using UPI is 0.02%, which amounts to 0.02 USD.
- After deducting the fee, the remaining amount is 99.98 USD.
- The exchange rate from USD to INR is 83.
- Therefore, the final converted amount is 99.98 * 83 = 8298.34 INR.


In [7]:
# Defining an agent to perform calculations using BuiltInCodeExecutor

calculator_agent = LlmAgent(
    name = "Calculator_agent",
    model = Gemini(model = "gemini-2.5-flash-lite", retry_options=retry_options),
    instruction= '''You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.''',
    code_executor=BuiltInCodeExecutor()
)

In [11]:
enhanced_currency_agent = LlmAgent(
    name = "Enhanced_currency_agent",
    model = Gemini(model = "gemini-2.5-flash-lite", retry_options=retry_options),
    instruction= """
You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.""",
    tools = [get_payment_method_charges, conversion_rate, AgentTool(agent=calculator_agent)]
)

In [12]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)


In [13]:
response = await enhanced_runner.run_debug("Convert 100 usd to inr using upi")


 ### Created new session: debug_session_id

User > Convert 100 usd to inr using upi


Enhanced_currency_agent > The final converted amount is 8299.98 INR.

Here's the breakdown of the calculation:

1.  **Transaction Fee**: The fee for using UPI is 0.0002% of the original amount.
    *   Fee percentage: 0.0002%
    *   Fee amount: 0.02 USD (100 USD \* 0.0002 / 100)
2.  **Amount After Fee**: The fee is deducted from the original amount.
    *   Amount remaining: 99.98 USD (100 USD - 0.02 USD)
3.  **Exchange Rate**: The conversion rate from USD to INR is 83.
4.  **Final Converted Amount**: The remaining amount is converted to INR using the exchange rate.
    *   Final amount: 8299.98 INR (99.98 USD \* 83)


In [16]:
show_python_code_and_result(response)

Generated Python Response >>  ```python
original_amount = 100
fee_percentage = 0.0002 / 100  # Convert percentage to decimal
exchange_rate = 83

fee_amount = original_amount * fee_percentage
amount_after_fee = original_amount - fee_amount
final_amount = amount_after_fee * exchange_rate

print(f'{final_amount=}')

```
```code_output
final_amount=8299.98
```
